In [1]:
import sys
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Different Paths
alg_path = "/workspaces/CP1-24-final/HeshamElsaman/Code/Algorithm"
data_path = "/workspaces/CP1-24-final/HeshamElsaman/Data/GPSSine"
sys.path.append(alg_path)
sys.path.append(data_path)

import final
from mercator_projection_functions import xy_on_earth

In [8]:
# Reloading the final module in case of changes taking place
importlib.reload(final)

<module 'final' from '/workspaces/CP1-24-final/HeshamElsaman/Code/Algorithm/final.py'>

In [6]:
folders_list = sorted(final.list_folders(data_path, "gps_sin"))
print(folders_list)
folders_dirs = [data_path + "/" + i for i in folders_list]
print(folders_dirs[0])

['gs01_gps_sin', 'gs02_gps_sin', 'gs03_gps_sin', 'gs04_gps_sin', 'gs05_gps_sin', 'gs06_gps_sin', 'gs07_gps_sin', 'gs08_gps_sin', 'gs09_gps_sin', 'gs10_gps_sin', 'gs11_gps_sin', 'gs12_gps_sin', 'gs13_gps_sin', 'gs14_gps_sin', 'gs15_gps_sin', 'gs16_gps_sin', 'gs17_gps_sin', 'gs18_gps_sin', 'gs19_gps_sin']
/workspaces/CP1-24-final/HeshamElsaman/Data/GPSSine/gs01_gps_sin


In [11]:
data_dict = {}

for i in folders_list:
    dir = data_path + "/" + i
    csv_path = dir + "/" + "Raw Data.csv"
    md_path = dir + "/" + i + ".md"
    x_list, y_list = final.xy_coord(csv_path)
    temp = final.read_temp(md_path)
    data_dict[i] = [temp, [x_list, y_list]]

print(data_dict)

{'gs01_gps_sin': [[np.float64(-8762.498652469963), np.float64(-8761.525796937258), np.float64(-8761.523516329311), np.float64(-8761.525233178982), np.float64(-8761.524331388124), np.float64(-8761.522240923505), np.float64(-8761.52091214413), np.float64(-8761.519838001139), np.float64(-8761.518249025637), np.float64(-8761.517089262552), np.float64(-8761.516230837718), np.float64(-8761.515164478371), np.float64(-8761.513687809747), np.float64(-8761.512347910879), np.float64(-8761.511451679771), np.float64(-8761.511476142656), np.float64(-8761.511520620626), np.float64(-8761.511667397928), np.float64(-8761.511265984243), np.float64(-8761.510525426032), np.float64(-8761.508972032907), np.float64(-8761.507510931571), np.float64(-8761.506750358272), np.float64(-8761.505844119622), np.float64(-8761.50499792623), np.float64(-8761.503855954332), np.float64(-8761.502745117015), np.float64(-8761.501347396787), np.float64(-8761.500219880232), np.float64(-8761.500346642448), np.float64(-8761.500751